In [61]:
import os
from langchain.chat_models import init_chat_model
os.environ['GROQ_API_KEY'] ='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'
llm = init_chat_model('groq:llama-3.1-8b-instant')

In [62]:
# start->router->general->respond
#         ->more info->        |->             |->checkHallucination->end
#         ->createPlan->conduct research->respond

In [63]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langgraph.graph import StateGraph,END,START
from langchain_core.tools import tool
from typing import TypedDict,Annotated,Union,List
from IPython.display import Image, display

In [64]:
class AgentState(TypedDict):
    input: str
    researchPlan: str
    additionalInfo: str
    researchResults: str
    webSearch: str
    retrievedDocuments: str
    response: str
    nextNode: str
    chatHistory: Annotated[List[HumanMessage],MessagesPlaceholder('chatHistory')]

In [65]:
llm = init_chat_model('groq:llama-3.1-8b-instant')

In [66]:
analysePrompt = ChatPromptTemplate.from_messages([
    ("system","""
    You are a great analyser that analyses the user queries for their category.The user query might be multiple sentence with sometimes clarification with them, consider them as 1 query. Analyse the user query and determine the apporpriate action:
    
    1.If the query is too vague, cannot be used to generate precise answer and require some context and clarification from user , respond: 'CLARIFY'
    2.If it is general information related query and not requires much research and you have enough input from user, respond:'GENERAl'.
    3.If it requires researching for information and analysis , respond: 'RESEARCH'.
    
    EXAMPLE
    'What is the weather today?' respond:'CLARIFY'
    'Help me with my homework.' respond:'CLARIFY'
    'Help me with my homework. what is 2+2' respond:'GENERAL'
    'What is the capital of france?' respond:'GENERAL'
    'Why does it snow?' respond:'GENERAL'
    'Write a report in quantum computing.' respond:'RESEARCH'
    'Progress in deep learning over the years.' respond:'RESEARCH'
    """),
    MessagesPlaceholder('chatHistory'),
    ("human","{input}"+". "+"{additionalInfo}")
])

In [67]:
generalPrompt = ChatPromptTemplate.from_messages([
    ("system","""Answer the user question precisely and clearly."""),
    MessagesPlaceholder("chatHistory"),
    ("human","{input}"+". "+"{additionalInfo}")
])

In [68]:
moreInfoPrompt = ChatPromptTemplate.from_messages([
    ("system","""Ask Clarifying question to better understand what user needs.Maximum questions is 3.
    Your output must only be a JSON object containing a single key 'queries' nothing else at all:
    {{ 'queries': ['Question 1', 'Question 2',...] }}"""),
    MessagesPlaceholder("chatHistory"),
    ("human","{input}"+". "+"{additionalInfo}")
])

In [69]:
createPlan = ChatPromptTemplate.from_messages([
    ("system","""Create a detailed research plan to answer the user queries."""),
    MessagesPlaceholder('chatHistory'),
    ("human","{input}"+". "+"{additionalInfo}")
])

In [70]:
# conductResearch = ChatPromptTemplate.from_messages([
#     ("system","""Conduct research based on this research plan:{researchPlan}"""),
#     MessagesPlaceholder('chatHistory'),
#     ("human","{input}"+". "+"{additionalInfo}")
# ])

In [71]:
def router(state:AgentState):
    chain = analysePrompt | llm
    result = chain.invoke({
        "input":state['input'],
        "chatHistory": state['chatHistory'],
        "additionalInfo":state['additionalInfo']
    })
    decision = result.content.lower()
    print("------------------------------------")
    print("DECISION=",decision)
    print("------------------------------------")
    
    if "general" in decision:
        print("GENERAL")
        return {"nextNode":"general"}
    
    elif "clarify" in decision:
        print("MOREINFO")
        return {"nextNode":"moreInfo"}

    else:
        print("RESEARCH")
        return {"nextNode":"createResearchPlan"}

In [72]:
def researchPlan(state:AgentState):
    
    chain = createPlan |llm
    result = chain.invoke({
        "input":state['input'],
        "chatHistory":state['chatHistory'],
        "additionalInfo":state['additionalInfo']
    })
    return {"researchPlan":result.content}

In [73]:
# def conductResearch(state:AgentState):
#     chain = conductResearch | llm
#     result = chain.invoke({
#         "input":state['input'],
#         "chatHistory":state['chatHistory'],
#         "researchPlan":state['researchPlan'],
#         "additionalInfo":state['additionalInfo']
#     })
#     return {'researchResult':result.content}

### Research Sub-graph

In [74]:
from langgraph.prebuilt import create_react_agent

In [75]:
from langchain_community.retrievers import ArxivRetriever

In [76]:
from langgraph.graph import add_messages

class ResearchState(TypedDict):
    query: str
    messages: Annotated[list,add_messages]

In [77]:
@tool
def Retrieve(query: str):
    """retrieve the arxiv for papers related to query"""
    all_papers=[]
    
    retriever = ArxivRetriever(
        load_max_docs=2,
        get_ful_documents=True,
    )
#     for query in nquery:
    docs = retriever.invoke(query)
    
    for doc in docs:
        paper_info = {
            "title": doc.metadata['Title'],
            "summary": doc.page_content,
            "authors": [author for author in doc.metadata['Authors'].split(',')],
            "published": doc.metadata['Published'].strftime("%Y-%m-%d"),
            "url": doc.metadata['Entry ID']
        }
        all_papers.append(paper_info)
        #limiting to one paper
        break


    return "Summary: "+all_papers[0]['summary']+"\n\n Source: "+all_papers[0]['url']

In [74]:
res = Retrieve("action classification using deep learning")
print(res)

Summary: Action parsing in videos with complex scenes is an interesting but
challenging task in computer vision. In this paper, we propose a generic 3D
convolutional neural network in a multi-task learning manner for effective Deep
Action Parsing (DAP3D-Net) in videos. Particularly, in the training phase,
action localization, classification and attributes learning can be jointly
optimized on our appearancemotion data via DAP3D-Net. For an upcoming test
video, we can describe each individual action in the video simultaneously as:
Where the action occurs, What the action is and How the action is performed. To
well demonstrate the effectiveness of the proposed DAP3D-Net, we also
contribute a new Numerous-category Aligned Synthetic Action dataset, i.e.,
NASA, which consists of 200; 000 action clips of more than 300 categories and
with 33 pre-defined action attributes in two hierarchical levels (i.e.,
low-level attributes of basic body part movements and high-level attributes
related to act

In [78]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key="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")

@tool
def WebSearch(query:str):
    """search the web for information related to query"""
    
    result = tavily_client.search(query,max_results=2)
    response = []
    
    for i in range(len(result['results'])):
        response.append(result['results'][i]['content'])
    return response[0]+"\n\n"+response[1]

In [147]:
li = WebSearch("who is leo messi?")
li

"Web oficial Leo Messi jugador del Inter de Miami – messi.com – Web oficial de Lionel Messi, jugador del Futbol Profesional y campeón mundial con la selección Argentina Web oficial Leo Messi jugador del Inter de Miami - messi.com Inter Miami CF disputa este domingo la novena jornada de la MLS ante Dallas en el Chase Stadium. Leo se enfrenta este sábado a Columbus Crew en la octava jornada de la MLS para Inter Miami en el […] Inter Miami y Chicago Fire empataron 0-0 en el partido correspondiente a la jornada 7 de la MLS del encuentro […] Leo Messi Management S.L.U. utiliza cookies propias y de terceros para ofrecerle contenidos adaptados a sus intereses.\n\nWidely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional footbal

In [39]:
print(response['results'][0]['content'])

"Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve Big Five league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals in a calendar year (91), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most assists (18) and goal contributions (32) in the Copa América, most goal contributions (21) in the World Cup, most international appearances (191) and international goals (112) by a South American male, and the second-most in the latter category outright."

In [79]:
tools=[WebSearch, Retrieve]

agent_system_prompt = """
< Role >
You are research assistant. You are a top-notch research assistant.Your task is to do research. 
</ Role >

< Tools >
You have access to the following tools:

1. WebSearch(query): search the web for information related to query
2. Retrieve(query): retrieve the arxiv for papers related to query

</ Tools >

< Instructions >
Using the tools available to gather information about the topic.
Do not return anything after generating last output.
Return last output as: {"webSearch":web_searched_data_here_exactly_as_retieved,"retrieve":arxiv_retireved_data_here_with_summary_and_source_exactly_as_retieved}
</ Instructions >
"""


In [80]:
def create_prompt(state):
    return [
        {
            "role": "system", 
            "content": agent_system_prompt
                
        }
    ] + state['messages']


In [133]:
subagent = create_react_agent(
    'groq:llama-3.1-8b-instant',
    tools=tools,
    prompt=create_prompt
)


In [196]:
response = subagent.invoke(
    {"messages": [{
        "role": "user", 
        "content": "Classification of user action using machine learning"
    }]},
#     config = RunnableConfig(recursion_limit=50)
)

In [197]:
response['messages'][-1].content

'{"webSearch": "Online classification of user activities using machine learning on network traffic - ScienceDirect","retrieve": "The current research has aimed to investigate and develop machine-learning approaches by using the data in the dataset to be applied to classify location-based social network data and predict user activities based on the nature of various locations (such as entertainment). The analysis of user activities and behavior from location-based social network data is often based on venue types, which\\nSummary: Machine learning has shown successes for complex learning problems in which\\ndata/parameters can be multidimensional and too complex for a first-principles\\nbased analysis. Some applications that utilize machine learning require human\\ninterpretability, not just to understand a particular result (classification,\\ndetection, etc.) but also for humans to take action based on that result.\\nBlack-box machine learning model interpretation has been studied, but

In [202]:
import json
input_str = json.loads(response['messages'][-1].content)
input_str

{'webSearch': 'Online classification of user activities using machine learning on network traffic - ScienceDirect',
 'retrieve': 'The current research has aimed to investigate and develop machine-learning approaches by using the data in the dataset to be applied to classify location-based social network data and predict user activities based on the nature of various locations (such as entertainment). The analysis of user activities and behavior from location-based social network data is often based on venue types, which\nSummary: Machine learning has shown successes for complex learning problems in which\ndata/parameters can be multidimensional and too complex for a first-principles\nbased analysis. Some applications that utilize machine learning require human\ninterpretability, not just to understand a particular result (classification,\ndetection, etc.) but also for humans to take action based on that result.\nBlack-box machine learning model interpretation has been studied, but rece

In [82]:
# # from langgraph.graph import StateGraph, START, END
# # from langgraph.types import Command
# # from typing import Literal
# # from IPython.display import Image, display
# research = StateGraph(ResearchState)
# research.add_node("response_agent", subagent)
# # research.add_edge(START, "response_agent")
# research.set_entry_point("response_agent")
# research.add_edge("response_agent",END)
# research = research.compile()


In [84]:
# display(Image(research.get_graph(xray=True).draw_mermaid_png()))

KeyboardInterrupt: 

In [87]:
response = research.invoke(
    {"messages": [{
        "role": "user", 
        "content": "computer vision"
    }]},
#     config = RunnableConfig(recursion_limit=50)
)

In [88]:
response['messages'][-1].content

'{"webSearch":"Computer vision tasks include methods for acquiring, processing, analyzing, and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the form of decisions.[1][2][3][4] \\"Understanding\\" in this context signifies the transformation of visual images (the input to the retina) into descriptions of the world that make sense to thought processes and can elicit appropriate action. Subdisciplines of computer vision include scene reconstruction, object detection, event detection, activity recognition, video tracking, object recognition, 3D pose estimation, learning, indexing, motion estimation, visual servoing, 3D scene modeling, and image restoration. Machine vision is the process of applying a range of technologies and methods to provide imaging-based automatic inspection, process control, and robot guidance[26] in industrial applications.[22] Machine vision tends to focus on a

In [91]:
response = research.invoke(
    {'query':"what is computer vision"}
)
response['messages'][-1].content

'{"webSearch": "Quantum Computing and its Applications", "retrieve": "Application of Quantum Computing What is Quantum Computing? How Quantum Computing is Different from Classical Computing? Quantum computing uses qubits to run quantum algorithms. The processing power of quantum computers can increase exponentially if more qubits are added. Like the processing power of quantum computers can increase exponentially if more qubits are added, similarly the processing power of classical computers increases linearly when more bits are added. Current State of Quantum Computing The field of quantum computing is evolving. Application of Quantum Computing Quantum computing can be used in various fields such as: Quantum Computing in Drug Discovery Quantum Computing in Cybersecurity Quantum Computing in Logistics Quantum Computing in Manufacturing Challenges and Limitations of Quantum Computing Future Prospects of Quantum Computing\\nThe discovery by Microsoft and Quantinuum addresses this problem

In [147]:
import json
def conductResearch(state:AgentState):
#     print("HERE1")
    response = subagent.invoke({
        "query":state['input']
    })
#     print("HERE2")
    try:
        webSearch = json.loads(response['messages'][-1].content)['webSearch']
        retrievedDocuments = json.loads(response['messages'][-1].content)['retrieve']
    except:
        print(response['messages'][-1].content)
        return {'webSearch':response['messages'][-1].content,'retrievedDocuments':response['messages'][-1].content,'nextNode':'respond'}
    return {'webSearch':webSearch,'retrievedDocuments':retrievedDocuments,'nextNode':'respond'}

In [135]:
def general(state:AgentState):
    
    chain = generalPrompt | llm
    result = chain.invoke({
        "input":state['input'],
        "chatHistory":state['chatHistory'],
        "additionalInfo":state['additionalInfo']
    })
#     print("GENERAL",result.content)
    return {'response':result.content}

In [153]:
import json
def moreInfo(state:AgentState):
    
    chain = moreInfoPrompt | llm
    result = chain.invoke({
        "input":state['input'],
        "chatHistory":state['chatHistory'],
        "additionalInfo":state['additionalInfo']
    })
#     print("MOREINFO:",result.content)
#     return {"response":result.content}
#     try:
#         queries = json.loads(result.content)['queries']
#         return {"response": "\n".join(queries)}
#     except:
#         return {"response": result.content}
    try:
        queries = json.loads(result.content)['queries']
        
    except:
        print(result.content)
        queries = result.content
    
    print("\nThe query is too vague.Answer these questions for better understanding:\n",queries)    
    additionalInfo = str(input("Enter:"))
    
    return {"additionalInfo":additionalInfo}

In [137]:
respondResearch = ChatPromptTemplate.from_messages([
    ("system","""Generate research report based on this research plan and retieved data:{researchPlan}
    Web Search: {webSearch}
    Retrieved Documents: {retirevedDocuments}"""),
    MessagesPlaceholder('chatHistory'),
    ("human","{input}"+". "+"{additionalInfo}")
])

In [138]:
def respond(state:AgentState):
#     print("RESPOND")
    if "researchResult" in state:
        return {"response": f"Research Findings:\n {state['researchResults']}"}
    else:
        return {}
#     print("RESPOND",response)
#     return {"response":response}

In [139]:
def respond(state:AgentState):
    chain = respondResearch | llm
    result = chain.invoke({
        "input":state['input'],
        "chatHistory":state['chatHistory'],
        "additionalInfo":state['additionalInfo'],
        "webSearch":state['webSearch'],
        "retirevedDocuments":state['retrievedDocuments'],
        "researchPlan":state['researchPlan']
    })
    return {'response':f"Research Result:\n {result.content}"}


In [154]:
graph = StateGraph(AgentState)

In [155]:
graph.add_node("Router",router)
graph.add_node("createResearchPlan",researchPlan)
graph.add_node("conductResearch",conductResearch)
graph.add_node("general",general)
graph.add_node("moreInfo",moreInfo)
graph.add_node("respond",respond)

graph.set_entry_point("Router")

# graph.add_edge("Router","createResearchPlan")
# graph.add_edge("Router","general")
graph.add_edge("createResearchPlan","conductResearch")
graph.add_edge("conductResearch","respond")
# graph.add_edge("Router","moreInfo")
graph.add_edge("respond",END)
graph.add_edge("general",END)
graph.add_edge("moreInfo","Router")

graph.add_conditional_edges(
    "Router",
    lambda state: state['nextNode'],
    {
        "general":"general",
        "createResearchPlan":"createResearchPlan",
        "moreInfo":"moreInfo"
    }
)

agent = graph.compile()




In [171]:
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

KeyboardInterrupt: 

In [143]:
def print_response(response):
    chatHistory = []
    print("\nAgent Response:")
    print(response["response"])
    chatHistory.extend([
        HumanMessage(content=response["input"]),
        HumanMessage(content=response["response"])
    ])

In [273]:
inpt = str(input("Enter your Query:"))
research_result = agent.invoke({
    "input":inpt,
    "chatHistory":[],
    "nextNode":None,
    "additionalInfo":None
})
print_response(research_result)

Enter your Query:help me with my homework
------------------------------------
DECISION= clarify
------------------------------------
MOREINFO

The query is too vague.Answer these questions for better understanding:
 ['Could you describe the topic of your homework so I can better assist you?', 'What kind of subject or subject is your homework about?', "Do you have any specific questions or areas where you're getting stuck?"]
Enter:mathematics. what is 2+4
------------------------------------
DECISION= general

i'll do my best to help with the math problem. the answer to 2 + 4 is 6. if you have any other math problems or need further assistance, feel free to ask.
------------------------------------
GENERAL

Agent Response:
The answer to 2 + 4 is 6.


In [274]:
research_result = agent.invoke({
        "input": "Classification of user actions using deep learning",
        "chatHistory": [],
        "nextNode": None,
        "additionalInfo":None
})
# print_response(research_result)
print_response(research_result)

------------------------------------
DECISION= clarify
------------------------------------
MOREINFO

The query is too vague.Answer these questions for better understanding:
 ['What are the different types of deep learning models used for classification?', 'Can you provide some examples of user action classification tasks?', 'How do I preprocess user action data for classification using deep learning?']
Enter:user actions like walking, running,etc
------------------------------------
DECISION= to classify user actions like walking, running, etc., using deep learning, we can use convolutional neural networks (cnns) or recurrent neural networks (rnns) depending on the type of input data. here's a general approach:

**input data:**

* we'll use video sequences or image sequences as input data.
* pre-process the data by:
  - resizing the images to a fixed size (e.g., 224x224).
  - normalizing the pixel values to the range [0, 1].
  - converting the videos to image sequences (frames).

**mo

In [109]:
inpt = str(input("Enter your Query:"))
research_result = agent.invoke({
    "input":inpt,
    "chatHistory":[],
    "nextNode":None,
    "additionalInfo":None
})
print_response(research_result)

Enter your Query:what is deep learning? write a report on it
------------------------------------
DECISION= research
------------------------------------
RESEARCH

Agent Response:
Research Result:
 **Deep Learning Report**

**Introduction**

Deep learning is a subset of machine learning that involves the use of artificial neural networks to analyze and interpret data. It is a type of machine learning that is inspired by the structure and function of the human brain, where layers of interconnected nodes (neurons) process and transmit information. Deep learning has gained significant attention in recent years due to its high accuracy and efficiency in solving complex problems.

**What is Deep Learning?**

Deep learning is a type of machine learning that uses neural networks with multiple layers to learn complex patterns and relationships in data. These neural networks are composed of multiple layers of nodes or "neurons," each of which applies a non-linear transformation to the input dat

In [152]:
##Final

In [151]:
inpt = str(input("Enter your Query:"))
research_result = agent.invoke({
    "input":inpt,
    "chatHistory":[],
    "nextNode":None,
    "additionalInfo":None
})
print_response(research_result)

Enter your Query:generate a report on machine learning
------------------------------------
DECISION= research
------------------------------------
RESEARCH

Agent Response:
Research Result:
 **Comprehensive Report on Machine Learning**

**Executive Summary**

Machine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, without being explicitly programmed. This report provides an in-depth understanding of machine learning, its history, types, applications, challenges, and future directions.

**Introduction**

Machine learning has revolutionized the way we approach problems in various fields, including computer vision, natural language processing, and predictive analytics. The ability of machines to learn from data has enabled them to make predictions, classify objects, and identify patterns, leading to significant improvements in efficiency and accuracy.

**History of Machine Learning**

Mach